<a href="https://colab.research.google.com/github/Mohit1053/quant-lab/blob/master/colab_E_optuna_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook E: Optuna Hyperparameter Sweep on NIFTY 500
**Run on Colab Pro+ H100** | Finds optimal hyperparameters for Transformer & TFT on broader universe

In [1]:
# === ENVIRONMENT SETUP ===
import subprocess, sys, os

# Reset working directory (prevents getcwd bug on re-run)
os.chdir("/content")

# Clean and re-clone for fresh code
if os.path.exists('/content/quant-lab'):
    print("Removing existing quant-lab directory...")
    subprocess.run(['rm', '-rf', '/content/quant-lab'])

print("Cloning repository...")
result = subprocess.run(
    ['git', 'clone', 'https://github.com/Mohit1053/quant-lab.git', '/content/quant-lab'],
    capture_output=True, text=True,
)
if result.returncode != 0:
    print(f"Clone failed: {result.stderr}")
    raise RuntimeError("Git clone failed")
print("Clone successful.")

os.chdir('/content/quant-lab')
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-e', '.'], check=True)
print("Package installed.")

# Install optuna
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'optuna'], check=True)
print("Optuna installed.")

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

from pathlib import Path
DRIVE_DIR = Path('/content/drive/MyDrive/quant_lab')
for d in ['data/features', 'data/cleaned', 'outputs/sweep/transformer', 'outputs/sweep/tft']:
    (DRIVE_DIR / d).mkdir(parents=True, exist_ok=True)

import torch
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu} ({mem:.1f} GB)")
else:
    print("WARNING: No GPU!")

Removing existing quant-lab directory...
Cloning repository...
Clone successful.
Package installed.
Optuna installed.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU: NVIDIA A100-SXM4-80GB (85.1 GB)


In [2]:
# === FIX NUMPY/SCIPY VERSIONS (Colab ships newer incompatible versions) ===
!pip uninstall -y numpy pandas scipy scikit-learn
!pip install --no-cache-dir numpy==1.26.4 pandas==2.2.2 scipy==1.11.4 scikit-learn==1.4.2

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: scipy 1.16.3
Uninstalling scipy-1.16.3:
  Successfully uninstalled scipy-1.16.3
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 143.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 306.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 138.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 101.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into acc

## Data Loading — Important Note

**NSE blocks Colab datacenter IPs**, so fetching NIFTY 500 tickers directly won't work.

**Recommended**: Upload `nifty500_features.parquet` (184MB) to Google Drive at `My Drive/quant_lab/data/features/` before running the next cell. Or run Notebook D first — it caches data to Drive.

In [2]:
# === LOAD NIFTY 500 DATA ===
# Priority: 1) Google Drive cache  2) File upload  3) NSE pipeline (may fail on Colab IPs)
import shutil
import pandas as pd

drive_features = DRIVE_DIR / 'data/features/nifty500_features.parquet'
local_features = Path('data/features/nifty500_features.parquet')
Path('data/features').mkdir(parents=True, exist_ok=True)
Path('data/cleaned').mkdir(parents=True, exist_ok=True)

loaded = False

# --- Option 1: Load from Google Drive (fastest, recommended for 184MB file) ---
if drive_features.exists():
    shutil.copy(drive_features, local_features)
    src = DRIVE_DIR / 'data/cleaned/nifty500_cleaned.parquet'
    if src.exists():
        shutil.copy(src, 'data/cleaned/nifty500_cleaned.parquet')
    print(f"✅ NIFTY 500 data loaded from Google Drive! ({drive_features.stat().st_size/1e6:.0f} MB)")
    loaded = True

# --- Option 2: Upload from local machine (may timeout for >100MB) ---
if not loaded:
    print("⚠️  No NIFTY 500 data on Drive at:")
    print(f"    {drive_features}")
    print()
    print("  RECOMMENDED: Upload nifty500_features.parquet to Google Drive:")
    print("    1. Go to drive.google.com")
    print("    2. Navigate to My Drive/quant_lab/data/features/")
    print("    3. Upload nifty500_features.parquet (184 MB)")
    print("    4. Re-run this cell")
    print()
    print("  Or try direct upload (may timeout for large files)...")
    try:
        from google.colab import files
        uploaded = files.upload()
        for fname, content in uploaded.items():
            if 'nifty500' in fname and fname.endswith('.parquet'):
                with open(str(local_features), 'wb') as f:
                    f.write(content)
                (DRIVE_DIR / 'data/features').mkdir(parents=True, exist_ok=True)
                shutil.copy(local_features, drive_features)
                print(f"✅ Uploaded {fname} ({len(content)/1e6:.1f} MB) — saved to Drive for next time!")
                loaded = True
                break
        if not loaded and uploaded:
            print("❌ No nifty500 parquet found in uploaded files.")
    except Exception as e:
        print(f"Upload skipped/failed: {e}")

# --- Option 3: Try NSE pipeline (usually fails from Colab) ---
if not loaded:
    print("\n⏳ Attempting NSE pipeline download (may fail if NSE blocks Colab IPs)...")
    try:
        subprocess.run(
            [sys.executable, 'scripts/run_pipeline.py', 'data.universe.name=nifty500'],
            check=True, timeout=600,
        )
        if local_features.exists():
            (DRIVE_DIR / 'data/features').mkdir(parents=True, exist_ok=True)
            shutil.copy(local_features, drive_features)
            print("✅ NIFTY 500 data downloaded and saved to Drive!")
            loaded = True
    except Exception as e:
        print(f"❌ NSE pipeline failed: {e}")
        print("   This is expected — NSE blocks Colab datacenter IPs.")
        print("   Upload nifty500_features.parquet to Google Drive and re-run this cell.")

if not loaded:
    raise FileNotFoundError(
        "Could not load NIFTY 500 data.\n"
        "Upload nifty500_features.parquet to Google Drive at:\n"
        f"  {drive_features}\n"
        "Or run Notebook D first to populate Google Drive."
    )

df = pd.read_parquet(local_features)
print(f"\nNIFTY 500: {df.shape[0]:,} rows, {df['ticker'].nunique()} tickers")

✅ NIFTY 500 data loaded from Google Drive! (193 MB)

NIFTY 500: 1,333,342 rows, 458 tickers


## Optuna Sweep — Transformer
TPE sampler with median pruning, 50 trials, 2h timeout

In [4]:
# === OPTUNA SWEEP: TRANSFORMER ===
import time
import json
import optuna
import numpy as np
import torch
from quant_lab.utils.seed import set_global_seed
from quant_lab.utils.device import get_device
from quant_lab.data.datasets import TemporalSplit
from quant_lab.data.datamodule import QuantDataModule, DataModuleConfig
from quant_lab.models.transformer.model import TransformerForecaster, TransformerConfig, MultiTaskLoss
from quant_lab.training.trainer import Trainer, TrainerConfig

set_global_seed(42)
device = get_device()

base_cols = {'date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'adj_close'}
feature_cols = [c for c in df.columns if c not in base_cols]
split = TemporalSplit(train_end='2021-12-31', val_end='2023-06-30')

def transformer_objective(trial):
    d_model = trial.suggest_categorical('d_model', [64, 128, 256])
    nhead = trial.suggest_categorical('nhead', [4, 8])
    num_layers = trial.suggest_int('num_encoder_layers', 2, 6)
    dim_ff = trial.suggest_categorical('dim_feedforward', [256, 512, 1024])
    dropout = trial.suggest_float('dropout', 0.05, 0.3)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    wd = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    seq_len = trial.suggest_categorical('sequence_length', [21, 42, 63])

    dm = QuantDataModule(
        df, feature_cols, split,
        DataModuleConfig(sequence_length=seq_len, target_col='log_return_1d', batch_size=batch_size, num_workers=2),
    )
    dm.setup()
    tl = dm.train_dataloader()
    vl = dm.val_dataloader()
    if tl is None:
        return float('inf')

    cfg = TransformerConfig(
        num_features=dm.num_features, d_model=d_model, nhead=nhead,
        num_encoder_layers=num_layers, dim_feedforward=dim_ff, dropout=dropout,
        direction_weight=0.3, volatility_weight=0.3,
    )
    model = TransformerForecaster(cfg)
    loss_fn = MultiTaskLoss(cfg)

    tc = TrainerConfig(
        epochs=30, learning_rate=lr, weight_decay=wd,
        warmup_steps=500, patience=5, mixed_precision=True,
        checkpoint_dir=f'outputs/sweep/transformer/trial_{trial.number}',
    )
    trainer = Trainer(model, loss_fn, tc, device)
    history = trainer.fit(tl, vl)
    if history['val_loss']:
        return min(history['val_loss'])
    return float('inf')


study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10),
)

print("Starting Transformer sweep (50 trials)...")
start = time.time()
study.optimize(transformer_objective, n_trials=50, timeout=7200)
elapsed = time.time() - start

print(f"\nSweep done in {elapsed/60:.1f} min")
print(f"Best val loss: {study.best_value:.6f}")
print(f"Best params: {json.dumps(study.best_params, indent=2)}")

# Save results
out_dir = Path('outputs/sweep/transformer')
out_dir.mkdir(parents=True, exist_ok=True)
with open(out_dir / 'best_params.json', 'w') as f:
    json.dump(study.best_params, f, indent=2)

trials_data = [{'number': t.number, 'value': t.value, 'params': t.params} for t in study.trials if t.value is not None]
with open(out_dir / 'all_trials.json', 'w') as f:
    json.dump(trials_data, f, indent=2)

# Save to Drive
for f_path in out_dir.glob('*.json'):
    shutil.copy(f_path, DRIVE_DIR / 'outputs/sweep/transformer' / f_path.name)
print("Transformer sweep results saved to Drive!")

2026-02-21 06:38:50 [info     ] using_gpu                      memory_gb=79.3 name='NVIDIA A100-SXM4-80GB'


[I 2026-02-21 06:38:50,488] A new study created in memory with name: no-name-d59272ed-7fb8-4894-b520-1ca53b0946c8


Streaming output truncated to the last 5000 lines.
2026-02-21 08:12:03 [debug    ] train_step                     loss=-2.609004 lr=3.78e-06 step=513950
2026-02-21 08:12:03 [debug    ] train_step                     loss=-2.970672 lr=3.77e-06 step=514000
2026-02-21 08:12:04 [debug    ] train_step                     loss=-2.943997 lr=3.77e-06 step=514050
2026-02-21 08:12:04 [debug    ] train_step                     loss=-2.944146 lr=3.77e-06 step=514100
2026-02-21 08:12:05 [debug    ] train_step                     loss=-3.045456 lr=3.77e-06 step=514150
2026-02-21 08:12:05 [debug    ] train_step                     loss=-2.741049 lr=3.77e-06 step=514200
2026-02-21 08:12:06 [debug    ] train_step                     loss=-3.093756 lr=3.77e-06 step=514250
2026-02-21 08:12:06 [debug    ] train_step                     loss=-2.926883 lr=3.77e-06 step=514300
2026-02-21 08:12:07 [debug    ] train_step                     loss=-2.725048 lr=3.77e-06 step=514350
2026-02-21 08:12:07 [debug    ]

[I 2026-02-21 08:56:41,870] Trial 0 finished with value: -3.0249739011633063 and parameters: {'d_model': 128, 'nhead': 4, 'num_encoder_layers': 2, 'dim_feedforward': 512, 'dropout': 0.22701814444901136, 'learning_rate': 1.0994335574766187e-05, 'weight_decay': 0.0008123245085588687, 'batch_size': 32, 'sequence_length': 63}. Best is trial 0 with value: -3.0249739011633063.



Sweep done in 137.9 min
Best val loss: -3.024974
Best params: {
  "d_model": 128,
  "nhead": 4,
  "num_encoder_layers": 2,
  "dim_feedforward": 512,
  "dropout": 0.22701814444901136,
  "learning_rate": 1.0994335574766187e-05,
  "weight_decay": 0.0008123245085588687,
  "batch_size": 32,
  "sequence_length": 63
}
Transformer sweep results saved to Drive!


## Optuna Sweep — TFT
TPE sampler with median pruning, 50 trials, 2h timeout

In [5]:
# === OPTUNA SWEEP: TFT ===
from quant_lab.models.tft.model import TFTForecaster, TFTConfig

def tft_objective(trial):
    d_model = trial.suggest_categorical('d_model', [16, 32, 64])
    nhead = trial.suggest_categorical('nhead', [2, 4])
    num_layers = trial.suggest_int('num_encoder_layers', 1, 3)
    lstm_hidden = trial.suggest_categorical('lstm_hidden', [16, 32, 64])
    grn_hidden = trial.suggest_categorical('grn_hidden', [8, 16, 32])
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    lr = trial.suggest_float('learning_rate', 1e-4, 1e-3, log=True)
    wd = trial.suggest_float('weight_decay', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    seq_len = trial.suggest_categorical('sequence_length', [21, 42, 63])

    dm = QuantDataModule(
        df, feature_cols, split,
        DataModuleConfig(sequence_length=seq_len, target_col='log_return_1d', batch_size=batch_size, num_workers=2),
    )
    dm.setup()
    tl = dm.train_dataloader()
    vl = dm.val_dataloader()
    if tl is None:
        return float('inf')

    cfg = TFTConfig(
        num_features=dm.num_features, d_model=d_model, nhead=nhead,
        num_encoder_layers=num_layers, lstm_layers=1, lstm_hidden=lstm_hidden,
        grn_hidden=grn_hidden, dropout=dropout,
        direction_weight=0.3, volatility_weight=0.3,
    )
    model = TFTForecaster(cfg)
    loss_cfg = TransformerConfig(num_features=dm.num_features, direction_weight=0.3, volatility_weight=0.3)
    loss_fn = MultiTaskLoss(loss_cfg)

    tc = TrainerConfig(
        epochs=30, learning_rate=lr, weight_decay=wd,
        warmup_steps=300, patience=5, mixed_precision=True,
        checkpoint_dir=f'outputs/sweep/tft/trial_{trial.number}',
    )
    trainer = Trainer(model, loss_fn, tc, device)
    history = trainer.fit(tl, vl)
    if history['val_loss']:
        return min(history['val_loss'])
    return float('inf')


tft_study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
)

print("Starting TFT sweep (50 trials)...")
start = time.time()
tft_study.optimize(tft_objective, n_trials=50, timeout=7200)
elapsed = time.time() - start

print(f"\nSweep done in {elapsed/60:.1f} min")
print(f"Best val loss: {tft_study.best_value:.6f}")
print(f"Best params: {json.dumps(tft_study.best_params, indent=2)}")

# Save
out_dir = Path('outputs/sweep/tft')
out_dir.mkdir(parents=True, exist_ok=True)
with open(out_dir / 'best_params.json', 'w') as f:
    json.dump(tft_study.best_params, f, indent=2)
trials_data = [{'number': t.number, 'value': t.value, 'params': t.params} for t in tft_study.trials if t.value is not None]
with open(out_dir / 'all_trials.json', 'w') as f:
    json.dump(trials_data, f, indent=2)

for f_path in out_dir.glob('*.json'):
    shutil.copy(f_path, DRIVE_DIR / 'outputs/sweep/tft' / f_path.name)
print("TFT sweep results saved to Drive!")

[I 2026-02-21 09:22:05,751] A new study created in memory with name: no-name-d3190cb3-c743-41ab-8173-6cff56bba40b


Streaming output truncated to the last 5000 lines.
2026-02-21 09:22:49 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=370 valid_samples=307
2026-02-21 09:22:49 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=370 valid_samples=307
2026-02-21 09:22:49 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=370 valid_samples=307
2026-02-21 09:22:49 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=370 valid_samples=307
2026-02-21 09:22:49 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=370 valid_samples=307
2026-02-21 09:22:49 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=370 valid_samples=307
2026-02-21 09:22:49 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=370 valid_samples=307
2026-02-21 09:22:49 [debug    ] 

[I 2026-02-21 12:23:32,052] Trial 0 finished with value: -3.0771453306303553 and parameters: {'d_model': 32, 'nhead': 2, 'num_encoder_layers': 1, 'lstm_hidden': 32, 'grn_hidden': 32, 'dropout': 0.4329770563201687, 'learning_rate': 0.00016305687346221485, 'weight_decay': 0.0002310201887845295, 'batch_size': 128, 'sequence_length': 63}. Best is trial 0 with value: -3.0771453306303553.



Sweep done in 181.4 min
Best val loss: -3.077145
Best params: {
  "d_model": 32,
  "nhead": 2,
  "num_encoder_layers": 1,
  "lstm_hidden": 32,
  "grn_hidden": 32,
  "dropout": 0.4329770563201687,
  "learning_rate": 0.00016305687346221485,
  "weight_decay": 0.0002310201887845295,
  "batch_size": 128,
  "sequence_length": 63
}
TFT sweep results saved to Drive!


## Summary
Best hyperparameters from both sweeps

In [6]:
print("=" * 60)
print("NOTEBOOK E COMPLETE — OPTUNA SWEEPS")
print("=" * 60)
print(f"\nTransformer best val loss: {study.best_value:.6f}")
print(f"Transformer best params: {json.dumps(study.best_params, indent=2)}")
print(f"\nTFT best val loss: {tft_study.best_value:.6f}")
print(f"TFT best params: {json.dumps(tft_study.best_params, indent=2)}")
print(f"\nAll results on Drive: {DRIVE_DIR / 'outputs/sweep'}")
print("=" * 60)

NOTEBOOK E COMPLETE — OPTUNA SWEEPS

Transformer best val loss: -3.024974
Transformer best params: {
  "d_model": 128,
  "nhead": 4,
  "num_encoder_layers": 2,
  "dim_feedforward": 512,
  "dropout": 0.22701814444901136,
  "learning_rate": 1.0994335574766187e-05,
  "weight_decay": 0.0008123245085588687,
  "batch_size": 32,
  "sequence_length": 63
}

TFT best val loss: -3.077145
TFT best params: {
  "d_model": 32,
  "nhead": 2,
  "num_encoder_layers": 1,
  "lstm_hidden": 32,
  "grn_hidden": 32,
  "dropout": 0.4329770563201687,
  "learning_rate": 0.00016305687346221485,
  "weight_decay": 0.0002310201887845295,
  "batch_size": 128,
  "sequence_length": 63
}

All results on Drive: /content/drive/MyDrive/quant_lab/outputs/sweep
